In [1]:
try:
    # Comment out if not using colab
    from google.colab import drive
    drive.mount('/content/drive')

    # Specific for luca's computer
    %cd "/content/drive/Othercomputers/lucas-yoga/Current/INFO381/code/INFO381-GitHub"
    using_colab = True
except:
    print("Not using Google Colab")
    using_colab = False

Mounted at /content/drive
/content/drive/Othercomputers/lucas-yoga/Current/INFO381/code/INFO381-GitHub


In [2]:
%pip install lime
%pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=41b749e3325a8960cc4017f6850e5030910315e4f5e0ab63315a1ef94ad38352
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tj7rjnzt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-tj7rjnzt
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn as nn
from sklearn.metrics import accuracy_score

import torch.nn.functional as F
from utils import get_dataloaders, cherry_pick_img_real, cherry_pick_img_ai_generated

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
# from torchcam.methods import GradCAM
# from torchcam.utils import overlay_mask
from torchvision.transforms.functional import to_pil_image

from lime import lime_image
from skimage.segmentation import mark_boundaries
import os

from classifiers import CLIPBinaryClassifier

In [4]:
train_loader, test_loader = get_dataloaders(batch_size=32, split = 'both')
print(test_loader)

Running in Google Colab


In [5]:
resnet_model = models.resnet18(pretrained = False)
num_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(num_ftrs, 2)
resnet_model.load_state_dict(torch.load("models/resnet18_cnn.pth"))
resnet_model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
correct = 0
total = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# Transform
resnet_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])

# Klassenavn
class_names = ['AI GENERATED', 'REAL']

In [8]:
clip_model = CLIPBinaryClassifier(model_path="models/clip_classifier_10epochs.pth", device=device)
clip_transform = clip_model.preprocessor

100%|███████████████████████████████████████| 338M/338M [00:09<00:00, 35.9MiB/s]


In [33]:
def get_predictions(full_path, true_label, model_1, model_2, transform_1, transform_2, class_names):
    try:
        img = Image.open(full_path).convert("RGB")
    except:
        print(f"Error opening {full_path}")
        return {
            "disagree":     False,
            "model1_fp":    False,
            "model1_fn":    False,
            "model2_fp":    False,
            "model2_fn":    False
        }
    input_tensor_1 = transform_1(img).unsqueeze(0).to(device)
    img_np_1 = transform_1(img).permute(1,2,0).cpu().numpy()

    input_tensor_2 = transform_2(img).unsqueeze(0).to(device)
    img_np_2 = transform_2(img).permute(1,2,0).cpu().numpy()

    with torch.no_grad():
        output_1 = model_1(input_tensor_1)
        output_2 = model_2(input_tensor_2)

        probs_1 = F.softmax(output_1, dim=1)
        pred_class_1 = torch.argmax(probs_1, dim=1).item()
        pred_label_1 = class_names[pred_class_1]

        probs_2 = F.softmax(output_2, dim=1)
        pred_class_2 = torch.argmax(probs_2, dim=1).item()
        pred_label_2 = class_names[pred_class_2]

    disagree = False
    if pred_label_1 != pred_label_2:
        disagree = True

    model1_fp = False
    model2_fp = False
    model1_fn = False
    model2_fn = False

    if pred_label_1 == "AI GENERATED" and true_label == 1:
        model1_fn = True
    if pred_label_1 == "REAL" and true_label == 0:
        model1_fp = True
    if pred_label_2 == "AI GENERATED" and true_label == 1:
        model2_fn = True
    if pred_label_2 == "REAL" and true_label == 0:
        model2_fp = True

    return {
        "disagree":     disagree,
        "model1_fp":    model1_fp,
        "model1_fn":    model1_fn,
        "model2_fp":    model2_fp,
        "model2_fn":    model2_fn
    }


In [34]:
import os
for i, j in enumerate(os.listdir("/content/fake_vs_real/test/fake")):
    path = f"/content/fake_vs_real/test/fake/{j}"
    pred = get_predictions(path, 0, resnet_model, clip_model, resnet_transform, clip_transform, class_names)
    if any(pred.values()):
        print(path)
        print(pred)
        print("*"*20)


/content/fake_vs_real/test/fake/16945.png
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/12527.jpg
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/24855.png
{'disagree': True, 'model1_fp': False, 'model1_fn': False, 'model2_fp': True, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/25554.png
{'disagree': True, 'model1_fp': False, 'model1_fn': False, 'model2_fp': True, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/3158.jpg
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/25741.png
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/183

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


/content/fake_vs_real/test/fake/15969.png
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/14800.png
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/1566.jpg
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/13267.png
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/16399.png
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/12406.jpg
{'disagree': True, 'model1_fp': True, 'model1_fn': False, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/fake/183

In [35]:
import os
for i, j in enumerate(os.listdir("/content/fake_vs_real/test/real")):
    path = f"/content/fake_vs_real/test/real/{j}"
    pred = get_predictions(path, 1, resnet_model, clip_model, resnet_transform, clip_transform, class_names)
    if any(pred.values()):
        print(path)
        print(pred)
        print("*"*20)

/content/fake_vs_real/test/real/8qa84fmkqp5a1.jpg
{'disagree': True, 'model1_fp': False, 'model1_fn': False, 'model2_fp': False, 'model2_fn': True}
********************
/content/fake_vs_real/test/real/1unvin8p726a1.png
{'disagree': False, 'model1_fp': False, 'model1_fn': True, 'model2_fp': False, 'model2_fn': True}
********************
/content/fake_vs_real/test/real/k8354ae1dy5a1.jpg
{'disagree': False, 'model1_fp': False, 'model1_fn': True, 'model2_fp': False, 'model2_fn': True}
********************
/content/fake_vs_real/test/real/49exuci4gj5a1.jpg
{'disagree': True, 'model1_fp': False, 'model1_fn': True, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/real/cpvsgdp1zj5a1.jpg
{'disagree': True, 'model1_fp': False, 'model1_fn': True, 'model2_fp': False, 'model2_fn': False}
********************
/content/fake_vs_real/test/real/ej2ykme2wl5a1.jpg
{'disagree': False, 'model1_fp': False, 'model1_fn': True, 'model2_fp': False, 'model2_fn': True}
*******